# AI Generated Characters for Learning and Wellbeing

Website: https://www.media.mit.edu/projects/ai-generated-characters/overview/

Paper: https://www.nature.com/articles/s42256-021-00417-9

Github: https://github.com/mitmedialab/AI-generated-characters


![](https://drive.google.com/uc?export=view&id=1jlJ820JYPhDyMhGnsplG9pOJdXeO_jKg)


This notebook is a combination of previous work on AI generated characters compiled into one easy to use pipeline that include [Siarohin et al.](https://github.com/AliaksandrSiarohin/first-order-model), [Prajwal et al.](https://github.com/Rudrabha/Wav2Lip), and [Corentin](https://github.com/CorentinJ/Real-Time-Voice-Cloning). Go check out their amazing work.

**Licensed under the MIT License**


Copyright (c) 2021 MIT Media Lab

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:
The above copyright notice and this permission notice shall be included in
all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN
THE SOFTWARE.

In [ ]:
#@markdown #**Check GPU type**
#@markdown ### Factory reset runtime if you don't have the desired GPU.

#@markdown ---




#@markdown V100 = Excellent (*Available only for Colab Pro users*)

#@markdown P100 = Very Good

#@markdown T4 = Good (*preferred*)

#@markdown K80 = Meh

#@markdown P4 = (*Not Recommended*) 

#@markdown ---

!nvidia-smi -L
#!nvcc --version

In [ ]:
#@markdown #**Installation of libraries**
# @markdown This cell will take a little while because it has to download several libraries.
%cd "/content" 
import requests

print("Downloading Packages")
# Character Images
!gdown --id "16HzQKA4e3vpLY8Em57WnE8UwIE591aF1" -O "/content/mona_lisa.png" &> /dev/null
!gdown --id "1cgfFgzm4BrqKIkyspGib6u4ty5ReyeM_" -O "/content/einstein.png" &> /dev/null
!gdown --id "10N3e5E0R1aYcLVmE_dmtMCSYVFGQLTeq" -O "/content/lincoln.png" &> /dev/null
!gdown --id "1-BeSNGGjJADs5W-Rn6izAteuVzJcnhW1" -O "/content/nietzsche.png" &> /dev/null
!gdown --id "1zPPUQ7xgbhnpVNl26J1Gl6rXlJ6g0rK7" -O "/content/sokrates.png" &> /dev/null
!gdown --id "1mzzEdXEOohLcpr8L01JzOVbirEMJogni" -O "/content/van_gogh.png" &> /dev/null

# Face Cropping
!wget "https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_frontalface_alt2.xml" -O "/content/haarcascade_frontalface_alt2.xml" &> /dev/null

# Wav2Lip
!git clone "https://github.com/Rudrabha/Wav2Lip.git"
!wget "https://www.adrianbulat.com/downloads/python-fan/s3fd-619a316812.pth" -O "Wav2Lip/face_detection/detection/sfd/s3fd.pth" &> /dev/null
!gdown --id "1IKhxXy0mplOpGFWLH9_uUhBoIplao8j0" -O "/content/Wav2Lip/checkpoints/wav2lip_gan.pth" &> /dev/null

# First-Order-Model
!git clone "https://github.com/AliaksandrSiarohin/first-order-model"
!gdown --id "19d9ZJYAMsNNQZd4AzIWCw4sF1EaNYuJ3" -O "/content/first-order-model/vox-cpk.pth.tar" &> /dev/null

# Template Data
#!gdown --id "1Qod7I5hiK1nCPsHBqAdK6hoYZgNzQPHi" -O "driving_video_long.mp4"
!gdown --id "1o2zD5xky8F6wZ21PkeG5KhJOlSdkeEpm" -O "driving_video.mp4" &> /dev/null

# Watermark
url = 'https://raw.githubusercontent.com/mitmedialab/AI-generated-characters/main/gen.png'
r = requests.get(url, allow_redirects=True) 
open('gen.png', 'wb').write(r.content)

# Noise
url = 'https://raw.githubusercontent.com/mitmedialab/AI-generated-characters/main/noise2.jpg'
r = requests.get(url, allow_redirects=True)
open('noise_2.png', 'wb').write(r.content)


print("Installing required libraries")
!pip install -r Wav2Lip/requirements.txt -y &> /dev/null
!pip uninstall tensorflow tensorflow-gpu -y &> /dev/null
!pip install ffmpeg -y &> /dev/null


# General Functions
print("Loading Libraries and functions")
import sys
from io import StringIO
from IPython import get_ipython

class IpyExit(SystemExit):
    """
    Exit Exception for IPython.
    Exception temporarily redirects stderr to buffer.
    """
    def __init__(self):
        print("Error: Please only select one input. If you will not use text please leave text field empty.")
        sys.stderr = StringIO()

    def __del__(self):
        sys.stderr.close()
        sys.stderr = sys.__stderr__  # restore from backup

from google.colab import files
def getLocalFiles():
  uploaded = files.upload()
  filename = next(iter(uploaded))
  return filename


# First-order-model
import imageio
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
import warnings
warnings.filterwarnings("ignore")

def trim_img(img_src):
  
  import imutils

  # Read the Input Image
  img = cv2.imread(img_src)
  img = imutils.resize(img, width=400)  

  # Convert into grayscale
  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

  # Trim to 400x400
  face_cascade = cv2.CascadeClassifier('/content/haarcascade_frontalface_alt2.xml')
  faces = face_cascade.detectMultiScale(gray, 1.1, 4)
  try:
    for (x, y, w, h) in faces:
      extention = 40
      faces = img[y-extention:y + h+extention, x-extention:x + w + extention]
      cv2.imwrite('/content/img_trimmed.png', faces)
  except:
    print("Error: Face takes too much space on image. Try a different image, or trim it yourself to 400x400.")

  return "/content/img_trimmed.png"


def animate_video(img_filename, vid_filename):
    %cd /content/first-order-model/
    
    from demo import make_animation
    from demo import load_checkpoints
    from skimage import img_as_ubyte

    source_image = imageio.imread(img_filename)
    driving_video = imageio.mimread(vid_filename, fps=30, memtest=False) 

    # Resize image and video to 256x256
    source_image = resize(source_image, (256, 256))[..., :3]
    driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]

    # Load Model
    generator, kp_detector = load_checkpoints(config_path='config/vox-256.yaml', checkpoint_path='/content/first-order-model/vox-cpk.pth.tar')

    # Make Animation
    predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=True,
                                adapt_movement_scale=False)
    #save resulting video
    imageio.mimsave('/content/vidvid.mp4', [img_as_ubyte(frame) for frame in predictions], fps=30)

    %cd /content


def tracability(video_filename):
  import moviepy.editor as mp

  video = mp.VideoFileClip(video_filename)

  machine = (mp.ImageClip('/content/noise_2.png')
    .set_duration(video.duration)
    .set_opacity(.05)
    .resize(height = 552) #
    .margin(right = 0, top = 0, opacity = 1.0)
    .set_pos(("center", "center")))
  
  human = (mp.ImageClip('/content/gen.png')
   .set_duration(video.duration)
   .resize(height = 50) #
   .margin(right = 0, top = 0, opacity = 1.0)
   .set_pos(("left", "bottom")))

  final = mp.CompositeVideoClip([video, machine, human])
  final.write_videofile("/content/marked.mp4")

print("Succesfully Finished Installing Libraries")

fatal: destination path 'Wav2Lip' already exists and is not an empty directory.
fatal: destination path 'first-order-model' already exists and is not an empty directory.
Installing required libraries
Loading Libraries and functions
Succesfully Finished Installing Libraries


In [ ]:
#@markdown #**Choose Character**

# TO DO: Show Images of Characters one can choose.

# @markdown Choose the character which you want to animate. If you have any requests for new characters to animate, please let us know here: patpat@mit.edu
character = 'Mona Lisa' #@param ["Van Gogh", "Mona Lisa", "Einstein", "Lincoln", "Nietzsche", "Sokrates", "Upload Your Own"]
print(f"{character} selected.")

if character == "Upload Your Own":
  character_img = "/content/"+getLocalFiles()
  if cv2.imread(character_img).shape[0] != cv2.imread(character_img).shape[1]:
    print("Cropping uploaded image")
    character_img = trim_img(character_img)

else:
  character = character.lower().replace(" ", "_") # make lowercase and remove spacing
  character_img = "/content/"+character+".png"

In [ ]:
#@markdown #**Choose Inputs**
# @markdown Please select one of the available inputs. Leave the text field empty if you want to animate the character with audio or video.


#Welcome. Today we will learn about the Theory of Relativity. I first came up with this method when...
text = "" #@param {type:"string"}
audio = True #@param {type:"boolean"}
video = False #@param {type:"boolean"}

if text != "" and audio or text !="" and video or audio and video:
  raise IpyExit
  

if video:
  print("Please upload the video you wish to drive the animation with:\n")
  video_driver = "/content/"+getLocalFiles()

  #to do: make sure only supported video formats can be uploaded

elif audio:
  print("Please upload the audio you wish to drive the animation with:\n")
  audio_driver = "/content/"+getLocalFiles()
  video_driver = "/content/driving_video.mp4"

elif text:
  print("Text is currently unsupported but will be soon.. Please use either audio or video inputs for now.")

In [ ]:
from numpy.core import memmap
import shutil


#@markdown #**Generate Character**
#@markdown This is likely to take a while depending on the length of your driving video. First we generate the movements of the character using the first-order-model approach, and then, if audio or text was given as input, we either synthesize audio from or use the audio provided to make the character lipsymc it using Wav2Lip.
!cd /content/
print("Animating Character with Driving Video: This might take a few minutes..")
animate_video(character_img, video_driver) # variables are only for showing HTML video
final_video_driver = "/content/vidvid.mp4"

if text != "":
  print("Generating speech from text")
  # generate audio
  #audio_driver = _GENERATED AUDIO.wav_
  audio = True

if audio:
  print("Lipsyncing Character with Audio")
  # Using Wav2Lip
  %cd /content/Wav2Lip
  !python inference.py --checkpoint_path "/content/Wav2Lip/checkpoints/wav2lip_gan.pth" --face $final_video_driver --audio $audio_driver &> /dev/null
  %cd /content
  final_video_driver = "/content/Wav2Lip/results/result_voice.mp4"
else:
  audio_driver = "/content/driver.wav"
  !ffmpeg -i $video_driver -q:a 0 -map 0:a "/content/driver.wav" -y &> /dev/null
  !ffmpeg -i $final_video_driver -i $audio_driver -c:v copy -c:a aac merged.mp4 -y &> /dev/null
  final_video_driver = "merged.mp4"

# Traceability
tracability(final_video_driver)
final_video_driver = "marked.mp4"
!ffmpeg -i $final_video_driver -i $audio_driver final_generated.mp4 -y &> /dev/null
!ffmpeg -i $final_video_driver ai_generated_character.mp4 -y &> /dev/null
final_video_driver = "ai_generated_character.mp4"

# display result
from IPython.display import HTML
from base64 import b64encode
mp4 = open("/content/final_generated.mp4",'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

In [ ]:
#@markdown ### **Download the Generated Video**
#@markdown Run this cell to download your generated video.

from google.colab import files
files.download(final_video_driver)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>